In [21]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Dropout
from sklearn.preprocessing import scale
from sklearn.metrics import roc_auc_score

features = pd.read_csv('train/train_small.csv', header = 0)
X = features.drop(['order_id','cancel', 'dow'], axis=1)
y = features['cancel']
print('{0} train samples loaded'.format(len(X)))
print(X.shape)

51761 train samples loaded
(51761, 32)


In [22]:
def fill_na_mean(df, col_names):
    for c in col_names:
        df[c] = df[c].fillna(df[c].mean())
    return df

X = fill_na_mean(X, ['shifts_num', 'dow_paid_share'])
X = X.fillna(0)
X = scale(X)

In [37]:
features = pd.read_csv('test/test-20-29-jun.csv', header = 0)

X_test = features.drop(['order_id','cancel', 'dow'], axis=1)

y_test = features['cancel']

X_test = fill_na_mean(X_test, ['shifts_num', 'dow_paid_share'])

X_test = X_test.fillna(0)
X_test = scale(X_test)

#y_pred = model.predict(X_test)

In [55]:
# import keras.backend as K
#
#def roc_auc(y_true, y_pred):
#    score = roc_auc_score(y_true, y_pred)
#    return K.abs(score)



model = Sequential()
#model.add(Dense(64, input_dim=32, W_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dense(64, input_dim=32, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X, y,
          epochs=30,
          batch_size=128,
          validation_data=(X_test, y_test))

#score = model.evaluate(x_test, y_test, batch_size=128)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_29 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 1)                 65        
Total params: 2,177
Trainable params: 2,177
Non-trainable params: 0
_________________________________________________________________
None
Train on 51761 samples, validate on 10742 samples
Epoch 1/30
51761/51761 [==============================] - 2s - loss: 0.5863 - acc: 0.7003 - val_loss: 0.5399 - val_acc: 0.7366
Epoch 2/30
51761/51761 [==============================] - 1s - loss: 0.5441 - acc: 0.7342 - val_loss: 0.5281 - val_acc: 0.7446
Epoch 3/30
51761/51761 [==============================] - 1s - loss: 0.5362 - acc: 0.7383 - val

In [50]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.499437618291
Test accuracy: 0.759076522107


In [57]:
features = pd.read_csv('test/test-20-29-jun.csv', header = 0)

X_test = features.drop(['order_id','cancel', 'dow'], axis=1)

y_true = features['cancel']

X_test = fill_na_mean(X_test, ['shifts_num', 'dow_paid_share'])

X_test = X_test.fillna(0)
X_test = scale(X_test)

y_pred = model.predict(X_test)

print(roc_auc_score(y_true, y_pred))

0.813215320039
